In [1]:
import tensorflow as tf
from transformers import BertTokenizer, TFBertModel, pipeline
from tensorflow.keras.layers import Input, Dense, Dropout
from tensorflow.keras.models import Model
from sklearn.model_selection import train_test_split
import pandas as pd


2024-01-17 14:10:44.953626: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2024-01-17 14:10:45.264290: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-01-17 14:10:45.264499: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-01-17 14:10:45.265595: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-01-17 14:10:45.628689: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2024-01-17 14:10:45.636401: I tensorflow/core/platform/cpu_feature_guard.cc:182] This Tens

In [2]:
df = pd.read_csv('./archive/train.csv')

In [3]:
def bert_encode(texts, tokenizer, max_len=512):
    all_tokens = []
    all_masks = []
    all_segments = []

    for text in texts:
        text = tokenizer.tokenize(text)
        text = text[:max_len-2]
        input_sequence = ["[CLS]"] + text + ["[SEP]"]
        pad_len = max_len - len(input_sequence)

        tokens = tokenizer.convert_tokens_to_ids(input_sequence)
        tokens += [0] * pad_len
        pad_masks = [1] * len(input_sequence) + [0] * pad_len
        segment_ids = [0] * max_len

        all_tokens.append(tokens)
        all_masks.append(pad_masks)
        all_segments.append(segment_ids)

    return np.array(all_tokens), np.array(all_masks), np.array(all_segments)


In [4]:
sentiment_pipeline = pipeline('sentiment-analysis')

def get_sentiment_score(sentence):
    return sentiment_pipeline(sentence)[0]['score']

No model was supplied, defaulted to distilbert-base-uncased-finetuned-sst-2-english and revision af0f99b (https://huggingface.co/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.
config.json: 100%|██████████| 629/629 [00:00<00:00, 804kB/s]
model.safetensors: 100%|██████████| 268M/268M [01:50<00:00, 2.43MB/s] 
2024-01-17 14:12:48.267975: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 93763584 exceeds 10% of free system memory.
2024-01-17 14:12:48.472789: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 93763584 exceeds 10% of free system memory.
2024-01-17 14:12:48.509656: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 93763584 exceeds 10% of free system memory.
2024-01-17 14:12:50.846029: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 93763584 exceeds 10% of free system memory.
2024-01-17 14:12:51.596398: W tensorf

In [14]:
print(get_sentiment_score("hello"))

0.9995185136795044
CPU times: user 516 ms, sys: 13.3 ms, total: 529 ms
Wall time: 509 ms


In [ ]:
# Prepare the data for BERT
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
max_len = 128  # You can change this as per your requirement
X = bert_encode(df['text_column_name'].values, tokenizer, max_len=max_len)
y = df['label_column_name'].values  # Replace 'label_column_name' with your label column name

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
# Model architecture
input_ids = Input(shape=(max_len,), dtype='int32')
attention_masks = Input(shape=(max_len,), dtype='int32')
bert_model = TFBertModel.from_pretrained('bert-base-uncased')
bert_output = bert_model(input_ids, attention_mask=attention_masks)[1]
drop = Dropout(0.3)(bert_output)
output = Dense(1, activation='sigmoid')(drop)
model = Model(inputs=[input_ids, attention_masks], outputs=output)

In [ ]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [ ]:
history = model.fit([X_train.input_ids, X_train.attention_masks], y_train, validation_split=0.1, epochs=5, batch_size=16)